# Preprocessing and Training Data for Arduino

## Preprocessing

In [54]:
import pandas as pd
Water_Dataset=pd.read_csv('dataset_final.csv')

In [55]:
Water_Dataset

,Suhu,pH,Turbidity,target
0,0.670823,0.084142,0.000000,0.0
1,0.563591,0.067961,0.033731,0.0
2,0.501247,0.103560,0.168972,0.0
3,0.593516,0.110032,0.153731,0.0
4,0.329177,0.000000,0.123003,0.0
...,...,...,...,...
85,0.992519,0.689320,0.399811,2.0
86,0.795511,0.760518,0.563150,2.0
87,0.882793,0.608414,0.439728,2.0
88,0.957606,0.588997,0.550674,2.0


In [56]:
Water_Dataset.describe()

,Suhu,pH,Turbidity,target
count,90.000000,90.000000,90.000000,90.000000
mean,0.668135,0.509241,0.315446,1.000000
std,0.237858,0.314053,0.239755,0.821071
min,0.000000,0.000000,0.000000,0.000000
25%,0.510599,0.131068,0.099062,0.000000
50%,0.679551,0.613269,0.274419,1.000000
75%,0.872818,0.773463,0.471266,2.000000
max,1.000000,1.000000,1.000000,2.000000


In [57]:
Water_Dataset.columns

Index(['Suhu', 'pH', 'Turbidity', 'target'], dtype='object')

In [58]:
Water_Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Suhu       90 non-null     float64
 1   pH         90 non-null     float64
 2   Turbidity  90 non-null     float64
 3   target     90 non-null     float64
dtypes: float64(4)
memory usage: 2.9 KB


In [59]:
Water_Dataset.shape

(90, 4)

In [60]:
Water_Dataset['target'].unique()

array([0., 1., 2.])

In [61]:
Water_Dataset.describe()

,Suhu,pH,Turbidity,target
count,90.000000,90.000000,90.000000,90.000000
mean,0.668135,0.509241,0.315446,1.000000
std,0.237858,0.314053,0.239755,0.821071
min,0.000000,0.000000,0.000000,0.000000
25%,0.510599,0.131068,0.099062,0.000000
50%,0.679551,0.613269,0.274419,1.000000
75%,0.872818,0.773463,0.471266,2.000000
max,1.000000,1.000000,1.000000,2.000000


# Train Model

## SVC

In [62]:
x = Water_Dataset.drop(['target'], axis = 1)
y = Water_Dataset['target']

In [64]:
x = Water_Dataset.drop(['target'], axis = 1)
y = Water_Dataset['target']

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [66]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(72, 3)
(18, 3)
(72,)
(18,)


In [71]:
model = SVC(C=1000, kernel='rbf', gamma=0.001)
model.fit(x_train,y_train)
pred = model.predict(x_test)
print("accuarcy score is : ", accuracy_score(y_test, pred))
print("classification report is : ", classification_report(y_test, pred))
print("confusion matirx is : ", confusion_matrix(y_test, pred))

accuarcy score is :  1.0
classification report is :                precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         7
         1.0       1.00      1.00      1.00         7
         2.0       1.00      1.00      1.00         4

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18

confusion matirx is :  [[7 0 0]
 [0 7 0]
 [0 0 4]]


In [95]:
# Create a Random Forest Classifier
model_rf = RandomForestClassifier(n_estimators=10)  # You can adjust the number of estimators

# Train the model on the training data
model_rf.fit(x_train, y_train)

# Make predictions on the test data
pred_rf = model_rf.predict(x_test)

# Evaluate the Random Forest Classifier's performance
print("Accuracy score is:", accuracy_score(y_test, pred_rf))
print("Classification report is:", classification_report(y_test, pred_rf))
print("Confusion matrix is:", confusion_matrix(y_test, pred_rf))

Accuracy score is: 1.0
Classification report is:               precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         4

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18

Confusion matrix is: [[7 0 0]
 [0 7 0]
 [0 0 4]]


# Training Data and Port to C code

In [74]:
import pandas as pd
Water_Dataset=pd.read_csv('dataset_final.csv')

## SVC

In [82]:
from micromlgen import port
from sklearn.svm import SVC


if __name__ == '__main__':
    X = Water_Dataset.drop(['target'], axis = 1)
    y = Water_Dataset['target']
    clf = SVC(C=1000, kernel='rbf', gamma=0.001).fit(X, y)
    classmap = {
        0: 'Bersih',
        1: 'Perlu Diencerkan',
        2: 'Perlu Dikuras'
    }
    c_code = port(clf, classmap=classmap)

    with open('classifier_svm.h', 'w') as file:
        file.write(c_code)